# Trabalho 1 - EEL891

Etapas que serão realizadas a seguir:
- Implementar o pré-processamento
- Visualizar, analisar e selecionar os atributos
- Escolher os métodos que serão usados e compará-los
- Verificar o desempenho com validação cruzada
- Ajustas os hiperparâmetros para chegar ao melhor resultado possível
- Treinar o modelo com o conjunto de teste

Antes de tudo, irei importar todas as bibliotecas necessárias:

In [2]:
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler

Agora, o próximo passo é importar o conjunto de treinamento:

In [3]:
dados = pd.read_csv(r"C:\Users\andre\Documents\UFRJ\SÉTIMO PERÍODO\Introdução ao Aprendizado de Máquina\EEL891-Trabalho-1\conjunto_de_treinamento.csv")
dados = dados.iloc[:,:]

Embaralhamento dos dados e separá-los em atributos e alvo (convertendo para um array):

In [4]:
dados = dados.sample(frac=1)
x = dados.iloc[:,:-1].to_numpy()
y = dados.iloc[:,-1].to_numpy()

Descartando algumas informações que não são relevantes. O campo "id_solicitante" é um número de 1 a 20000, ou seja, apenas atrapalharia o modelo. Os campos "grau_instrucao" e "possui_telefone_celular" não foram extraídos corretamente, então segui a orientação do professor para descartá-los. E o campo "qtde_contas_bancarias_especiais" é exatamente igual ao campo "qtde_contas_bancarias", logo foi descartado seguindo a orientação do professor.

In [5]:
dados = dados.drop(['id_solicitante', 'grau_instrucao', 'possui_telefone_celular', 'qtde_contas_bancarias_especiais'],axis=1)

Identificação das variáveis categóricas olhando o dicionário de dados:

In [6]:
variaveisCategoricas = [x for x in dados.columns if dados[x].dtype=='object' or x == 'produto_solicitado' or x == 'tipo_endereco' or x =='estado_civil' or  x =='nacionalidade' or x=='tipo_residencia' or x=='possui_email' or x=='possui_cartao_visa' or x=='possui_cartao_mastercard' or x=='possui_cartao_diners' or x=='possui_cartao_amex' or x=='possui_cartao_cartoes' or x=='possui_carro' or x=='profissao' or x=='ocupacao' or x=='profissao_companheiro' or x=='local_onde_reside' or x=='local_onde_trabalha']
print(variaveisCategoricas)

['produto_solicitado', 'forma_envio_solicitacao', 'tipo_endereco', 'sexo', 'estado_civil', 'nacionalidade', 'estado_onde_nasceu', 'estado_onde_reside', 'possui_telefone_residencial', 'codigo_area_telefone_residencial', 'tipo_residencia', 'possui_email', 'possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_carro', 'vinculo_formal_com_empresa', 'estado_onde_trabalha', 'possui_telefone_trabalho', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao', 'profissao_companheiro', 'local_onde_reside', 'local_onde_trabalha']


Verificando as variáveis categóricas (quantidade de categorias e quais são):

In [7]:
for v in variaveisCategoricas:
    print("\n%15s: " %v, "%4d categorias" % len(dados[v].unique() ) )
    print(dados[v].unique(), "\n")


produto_solicitado:     3 categorias
[2 1 7] 


forma_envio_solicitacao:     3 categorias
['correio' 'presencial' 'internet'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     4 categorias
['M' 'F' 'N' ' '] 


   estado_civil:     8 categorias
[2 1 5 0 4 3 6 7] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['BA' 'MT' 'RS' 'AL' 'CE' ' ' 'SP' 'GO' 'PE' 'MS' 'PB' 'PA' 'PR' 'MG' 'RJ'
 'MA' 'RN' 'PI' 'SC' 'ES' 'AC' 'SE' 'AM' 'TO' 'AP' 'RO' 'RR' 'DF'] 


estado_onde_reside:    27 categorias
['BA' 'MT' 'SP' 'CE' 'MG' 'RS' 'PA' 'PE' 'RO' 'MS' 'PB' 'PI' 'GO' 'RJ'
 'RN' 'AP' 'AL' 'DF' 'AC' 'PR' 'SE' 'MA' 'ES' 'AM' 'SC' 'TO' 'RR'] 


possui_telefone_residencial:     2 categorias
['Y' 'N'] 


codigo_area_telefone_residencial:    81 categorias
['81' '71' '5' '84' '16' '86' '112' '29' ' ' '107' '120' '97' '77' '15'
 '103' '110' '66' '75' '54' '90' '62' '20' '58' '12' '105' '22' '32' '61'
 '123' '100' '14' '111' '69' '76' '117' '13' '40' '91' '

Correção de alguns valores (principalmente em branco). A ideia inicial é criar uma nova categoria para os valores em branco, e depois tentar excluí-los para ver qual é a resposta dos modelos:

In [37]:
for x in variaveisCategoricas:
    if x == 'sexo':
        dados[x].replace(' ', 'N', inplace=True)
    if x == 'estado_onde_nasceu':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_residencial':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'tipo_residencia':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'estado_onde_trabalha':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_trabalho':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'profissao':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'ocupacao':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'profissao_companheiro':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")


produto_solicitado:     3 categorias
[2 1 7] 


forma_envio_solicitacao:     3 categorias
['correio' 'presencial' 'internet'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     3 categorias
['M' 'F' 'N'] 


   estado_civil:     8 categorias
[2 1 5 0 4 3 6 7] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['BA' 'MT' 'RS' 'AL' 'CE' 'NN' 'SP' 'GO' 'PE' 'MS' 'PB' 'PA' 'PR' 'MG'
 'RJ' 'MA' 'RN' 'PI' 'SC' 'ES' 'AC' 'SE' 'AM' 'TO' 'AP' 'RO' 'RR' 'DF'] 


estado_onde_reside:    27 categorias
['BA' 'MT' 'SP' 'CE' 'MG' 'RS' 'PA' 'PE' 'RO' 'MS' 'PB' 'PI' 'GO' 'RJ'
 'RN' 'AP' 'AL' 'DF' 'AC' 'PR' 'SE' 'MA' 'ES' 'AM' 'SC' 'TO' 'RR'] 


possui_telefone_residencial:     2 categorias
['Y' 'N'] 


codigo_area_telefone_residencial:    81 categorias
['81' '71' '5' '84' '16' '86' '112' '29' '000' '107' '120' '97' '77' '15'
 '103' '110' '66' '75' '54' '90' '62' '20' '58' '12' '105' '22' '32' '61'
 '123' '100' '14' '111' '69' '76' '117' '13' '40' '91' '4

In [39]:
print(variaveisCategoricas)

['produto_solicitado', 'forma_envio_solicitacao', 'tipo_endereco', 'sexo', 'estado_civil', 'nacionalidade', 'estado_onde_nasceu', 'estado_onde_reside', 'possui_telefone_residencial', 'codigo_area_telefone_residencial', 'tipo_residencia', 'possui_email', 'possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_carro', 'vinculo_formal_com_empresa', 'estado_onde_trabalha', 'possui_telefone_trabalho', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao', 'profissao_companheiro', 'local_onde_reside', 'local_onde_trabalha']


In [49]:
a=0
for x in dados['tipo_endereco']:
    if x == 1:
        a+=1
print(a)

127


In [42]:
# 127 em 20000 (tipo_endereco == 1)
binarizador = LabelBinarizer()
for x in ['tipo_endereco', 'possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho']:
    dados[x] = binarizador.fit_transform(dados[x])
print(dados.head(4).T)

                                     1338        1577        8157        4254
produto_solicitado                      2           1           1           2
dia_vencimento                         15          25          25          15
forma_envio_solicitacao           correio  presencial  presencial  presencial
tipo_endereco                           0           0           0           0
sexo                                    M           F           F           M
idade                                  45          21          28          44
estado_civil                            2           1           1           1
qtde_dependentes                        0           0           1           0
nacionalidade                           1           1           1           1
estado_onde_nasceu                     BA          MT          BA          RS
estado_onde_reside                     BA          MT          SP          BA
possui_telefone_residencial             1           1           

## Pré-processamento

Classificador Bayesiano com Distribuição Multinomial

In [ ]:
classificador = MultinomialNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

Classificador Bayesiano com Distribuição Multinomial

In [ ]:
classificador = GaussianNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))